In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import matplotlib.pyplot as plt
from tensorflow.keras import Sequential, layers

In [3]:
datagen = ImageDataGenerator()
train_path = os.path.join('chest_xray', 'train')
test_path = os.path.join('chest_xray', 'test')
val_path = os.path.join('chest_xray', 'val')

train_gen = datagen.flow_from_directory(train_path)
test_gen = datagen.flow_from_directory(test_path)
val_gen = datagen.flow_from_directory(val_path)

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


In [ ]:
len(train_gen.labels)

`model.fit_generator(train_gen, validation_data=val_gen)`

In [7]:
def visualize_training_results(results):
    history = results.history
    plt.figure()
    plt.plot(history['val_loss'])
    plt.plot(history['loss'])
    plt.legend(['val_loss', 'loss'])
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.show()
    
    plt.figure()
    plt.plot(history['val_acc'])
    plt.plot(history['acc'])
    plt.legend(['val_acc', 'acc'])
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.show()

In [19]:
def baseline_model(image_shape):
    model_layers = [
        layers.Input(image_shape),
        layers.Flatten(),
        layers.Dense(24, activation='relu'),
        layers.Dense(2, activation='sigmoid')
    ]
    model = Sequential(model_layers)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [22]:
visualize_training_results(baseline_model)

TypeError: 'History' object is not subscriptable

<Figure size 432x288 with 0 Axes>

In [20]:
baseline_model = baseline_model(train_gen.image_shape)
baseline_model.fit(train_gen, validation_data=val_gen, epochs=10)

Epoch 1/10
163/163 [==============================] - 69s 423ms/step - loss: 312.6325 - accuracy: 0.7339 - val_loss: 0.6943 - val_accuracy: 0.5000
Epoch 2/10
141/163 [========================>.....] - ETA: 9s - loss: 0.6590 - accuracy: 0.7440

KeyboardInterrupt: 

In [28]:
def conv_1(image_shape):
    these_layers = [
        layers.Input(image_shape),
        layers.Conv2D(32, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(2, activation='softmax')
    ]
    model = Sequential(these_layers)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [29]:
model_2 = conv_1(train_gen.image_shape)
model_2.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 254, 254, 32)      896       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 127, 127, 32)      0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 516128)            0         
_________________________________________________________________
dense_14 (Dense)             (None, 128)               66064512  
_________________________________________________________________
dense_15 (Dense)             (None, 2)                 258       
Total params: 66,065,666
Trainable params: 66,065,666
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model_2.fit(train_gen, validation_data=val_gen, epochs=10)

Epoch 1/10
163/163 [==============================] - 122s 744ms/step - loss: 304.8205 - accuracy: 0.9062 - val_loss: 2.8041 - val_accuracy: 1.0000
Epoch 2/10
 87/163 [===============>..............] - ETA: 55s - loss: 2.1048 - accuracy: 0.9795